In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import glob
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

%matplotlib inline
# For Filtering the warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('kidney_disease.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [5]:
data.classification=data.classification.replace("ckd\t","ckd")
data.classification.unique()

array(['ckd', 'notckd'], dtype=object)

In [6]:
data.drop('id', axis = 1, inplace = True)
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [7]:
data['classification'] = data['classification'].replace(['ckd','notckd'], [1,0])
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,1
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,1
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,1
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,1
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,1


In [8]:
df = data.dropna(axis = 0)
print(f"Before dropping all NaN values: {data.shape}")
print(f"After dropping all NaN values: {df.shape}")
df.head()

Before dropping all NaN values: (400, 25)
After dropping all NaN values: (158, 25)


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,1
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,29,12100,3.7,yes,yes,no,poor,no,yes,1
11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,380.0,...,32,4500,3.8,yes,yes,no,poor,yes,no,1
14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,157.0,...,16,11000,2.6,yes,yes,yes,poor,yes,no,1
20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,...,24,9200,3.2,yes,yes,yes,poor,yes,yes,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 3 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             158 non-null    float64
 1   bp              158 non-null    float64
 2   sg              158 non-null    float64
 3   al              158 non-null    float64
 4   su              158 non-null    float64
 5   rbc             158 non-null    object 
 6   pc              158 non-null    object 
 7   pcc             158 non-null    object 
 8   ba              158 non-null    object 
 9   bgr             158 non-null    float64
 10  bu              158 non-null    float64
 11  sc              158 non-null    float64
 12  sod             158 non-null    float64
 13  pot             158 non-null    float64
 14  hemo            158 non-null    float64
 15  pcv             158 non-null    object 
 16  wc              158 non-null    object 
 17  rc              158 non-null    obj

In [10]:
df['pcv']=df['pcv'].astype(int)
df['wc']=df['wc'].astype(int)
df['rc']=df['rc'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 3 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             158 non-null    float64
 1   bp              158 non-null    float64
 2   sg              158 non-null    float64
 3   al              158 non-null    float64
 4   su              158 non-null    float64
 5   rbc             158 non-null    object 
 6   pc              158 non-null    object 
 7   pcc             158 non-null    object 
 8   ba              158 non-null    object 
 9   bgr             158 non-null    float64
 10  bu              158 non-null    float64
 11  sc              158 non-null    float64
 12  sod             158 non-null    float64
 13  pot             158 non-null    float64
 14  hemo            158 non-null    float64
 15  pcv             158 non-null    int64  
 16  wc              158 non-null    int64  
 17  rc              158 non-null    flo

In [11]:
dictonary = {
        "rbc": {
        "abnormal":1,
        "normal": 0,
    },
        "pc":{
        "abnormal":1,
        "normal": 0,
    },
        "pcc":{
        "present":1,
        "notpresent":0,
    },
        "ba":{
        "notpresent":0,
        "present": 1,
    },
        "htn":{
        "yes":1,
        "no": 0,
    },
        "dm":{
        "yes":1,
        "no":0,
    },
        "cad":{
        "yes":1,
        "no": 0,
    },
        "appet":{
        "good":1,
        "poor": 0,
    },
        "pe":{
        "yes":1,
        "no":0,
    },
        "ane":{
        "yes":1,
        "no":0,
    }
}

In [12]:
df=df.replace(dictonary)

In [13]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
3,48.0,70.0,1.005,4.0,0.0,0,1,1,0,117.0,...,32,6700,3.9,1,0,0,0,1,1,1
9,53.0,90.0,1.020,2.0,0.0,1,1,1,0,70.0,...,29,12100,3.7,1,1,0,0,0,1,1
11,63.0,70.0,1.010,3.0,0.0,1,1,1,0,380.0,...,32,4500,3.8,1,1,0,0,1,0,1
14,68.0,80.0,1.010,3.0,2.0,0,1,1,1,157.0,...,16,11000,2.6,1,1,1,0,1,0,1
20,61.0,80.0,1.015,2.0,0.0,1,1,0,0,173.0,...,24,9200,3.2,1,1,1,0,1,1,1


In [14]:
countNoDisease = len(df[df['classification'] == 0])
countHaveDisease = len(df[df['classification'] == 1])
print("Percentage of Patients Haven't CKD Disease: {:.2f}%".format((countNoDisease / (len(df['classification']))*100)))
print("Percentage of Patients Have CKD Disease: {:.2f}%".format((countHaveDisease / (len(df['classification']))*100)))


Percentage of Patients Haven't CKD Disease: 72.78%
Percentage of Patients Have CKD Disease: 27.22%


In [15]:
from sklearn.model_selection import train_test_split
X = df.drop(['classification', 'sg', 'appet', 'rc', 'pcv', 'hemo', 'sod'], axis = 1)
Y = df['classification']

In [16]:
X.columns

Index(['age', 'bp', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu', 'sc',
       'pot', 'wc', 'htn', 'dm', 'cad', 'pe', 'ane'],
      dtype='object')

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assuming df contains your dataset
X = df.drop(['classification', 'sg', 'appet', 'rc', 'pcv', 'hemo', 'sod'], axis=1)
Y = df['classification']

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build the AdaBoost model with DecisionTreeClassifier as the base estimator
ada_boost_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                                     n_estimators=50, random_state=42)

# Train the model
ada_boost_model.fit(X_train, Y_train)

# Make predictions on the training set
Y_train_pred_ada_boost = ada_boost_model.predict(X_train)
train_accuracy_ada_boost = accuracy_score(Y_train, Y_train_pred_ada_boost)
print('Accuracy on Training data (AdaBoost):', train_accuracy_ada_boost)

# Make predictions on the test set
Y_test_pred_ada_boost = ada_boost_model.predict(X_test)
test_accuracy_ada_boost = accuracy_score(Y_test, Y_test_pred_ada_boost)
print('Accuracy on Test data (AdaBoost):', test_accuracy_ada_boost)


Accuracy on Training data (AdaBoost): 1.0
Accuracy on Test data (AdaBoost): 1.0


In [22]:
import numpy as np

# Assuming your original input_data with missing values
input_data = (24, 100, 2, 0, 1, 0, 1, 0, 136, 60, 1.9, 3.7, 9600, 1, 1, 0, 0, 1)

# Change the input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Impute missing values with the mean of the non-missing values
input_data_as_numpy_array = np.nan_to_num(input_data_as_numpy_array, nan=np.nanmean(input_data_as_numpy_array))

# Reshape the numpy array to match the expected shape for XGBoost prediction
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

# Assuming you have already defined and trained your AdaBoost model
prediction = ada_boost_model.predict(input_data_reshaped)

if prediction[0] == 0:
    print("The person does not have CKD disease.")
else:
    print("The person has CKD disease.")


The person has CKD disease.


Saving Trained Model


In [23]:
import pickle

In [24]:
filename='trained_model.sav'
pickle.dump(ada_boost_model,open(filename,'wb'))

In [25]:
#loading the saved model

In [26]:
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [28]:
import numpy as np

# Assuming your original input_data with missing values
input_data = (51, 0, 0, 0, 1, 0, 0, 0, 121, 27, 0.8, 3.7, 8300, 0, 0, 0, 0, 0)

# Change the input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Impute missing values with the mean of the non-missing values
input_data_as_numpy_array = np.nan_to_num(input_data_as_numpy_array, nan=np.nanmean(input_data_as_numpy_array))

# Reshape the numpy array to match the expected shape for XGBoost prediction
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

# Print the number of features in input_data_reshaped
print("Number of features in input_data_reshaped:", input_data_reshaped.shape[1])

# Assuming you have already defined and trained your XGBoost model
prediction = loaded_model.predict(input_data_reshaped)

if prediction[0] == 0:
    print("The person does not have CKD disease.")
else:
    print("The person has CKD disease.")


Number of features in input_data_reshaped: 18
The person does not have CKD disease.
